## Indox Retrieval Augmentation
Here, we will explore how to work with Indox Retrieval Augmentation. We are using GoogleAi from Indox , we should set our GOOGLE_API_KEY as an environment variable.

In [ ]:
!pip install -q -U google-generativeai
!pip install chromadb
!pip install indox

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [13]:
from indox import IndoxRetrievalAugmentation
from indox.llms import GoogleAi
from indox.embeddings import HuggingFaceEmbedding
from indox.data_loader_splitter import ClusteredSplit

### Creating an instance of IndoxTetrivalAugmentation

To effectively utilize the Indox Retrieval Augmentation capabilities, you must first create an instance of the IndoxRetrievalAugmentation class. This instance will allow you to access the methods and properties defined within the class, enabling the augmentation and retrieval functionalities.

In [ ]:
indox = IndoxRetrievalAugmentation()

In [3]:
google_qa = GoogleAi(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash-latest")
embed = HuggingFaceEmbedding(model="multi-qa-mpnet-base-cos-v1")

2024-06-30 18:08:07,830 INFO:IndoxRetrievalAugmentation initialized
2024-06-30 18:08:07,831 INFO:Initializing HuggingFaceModel with model: deepset/roberta-base-squad2
2024-06-30 18:08:07,831 INFO:HuggingFaceModel initialized successfully
2024-06-30 18:08:07,976 INFO:Initializing GoogleAi with model: gemini-1.5-flash-latest
2024-06-30 18:08:07,977 INFO:GoogleAi initialized successfully
2024-06-30 18:08:09,994 INFO:Load pretrained SentenceTransformer: multi-qa-mpnet-base-cos-v1
2024-06-30 18:08:11,455 INFO:Use pytorch device: cpu
2024-06-30 18:08:11,456 INFO:Initialized HuggingFace embeddings with model: multi-qa-mpnet-base-cos-v1


In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [4]:
file_path = "sample.txt"

## Data Loader Setup

We set up the data loader using the `ClusteredSplit` class. This step involves loading documents, configuring embeddings, and setting options for processing the text.


In [14]:
load_splitter = ClusteredSplit(file_path=file_path,summary_model=google_qa,embeddings=embed)
docs = load_splitter.load_and_chunk()

2024-06-30 18:11:37,375 INFO:Initializing ClusteredSplit
2024-06-30 18:11:37,375 INFO:ClusteredSplit initialized successfully
2024-06-30 18:11:37,376 INFO:Getting all documents
2024-06-30 18:11:37,376 INFO:Starting processing for documents
2024-06-30 18:11:47,798 INFO:Generating summary for documentation
2024-06-30 18:11:47,799 INFO:Generating response for prompt: You are a helpful assistant. Give a detailed summary of the documentation provided.

Documentation:
 ["The wife of a rich man fell sick, and as she felt that her end was drawing near, she called her only daughter to her bedside and said, dear child, be good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you   Thereupon she closed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came", 'the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off aga

--Generated 6 clusters--


2024-06-30 18:11:51,695 INFO:Response generated successfully
2024-06-30 18:11:51,696 INFO:Generating summary for documentation
2024-06-30 18:11:51,697 INFO:Generating response for prompt: You are a helpful assistant. Give a detailed summary of the documentation provided.

Documentation:
 ['  They took her pretty clothes away from her, put an old grey bedgown on her, and gave her wooden shoes   Just look at the proud princess, how decked out she is, they cried, and laughed, and led her into the kitchen There she had to do hard work from morning till night, get up before daybreak, carry water, light fires, cook and wash   Besides this, the sisters did her every imaginable injury - they mocked her', "prince approached her, took her by the hand and danced with her He would dance with no other maiden, and never let loose of her hand, and if any one else came to invite her, he said, this is my partner She danced till it was evening, and then she wanted to go home But the king's son said, I w

--Generated 1 clusters--


2024-06-30 18:12:48,494 INFO:Response generated successfully
2024-06-30 18:12:48,496 INFO:Completed chunking & clustering process
2024-06-30 18:12:48,496 INFO:Successfully obtained all documents


In [15]:
docs

["The wife of a rich man fell sick, and as she felt that her end was drawing near, she called her only daughter to her bedside and said, dear child, be good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you   Thereupon she closed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came",
 'the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench',
 '  They took her pretty clothes away from her, put an old grey bedgown on her, and gave her wooden shoes   Just look at the pr

## Vector Store Connection and Document Storage

In this step, we connect the Indox application to the vector store and store the processed documents.


In [6]:
from indox.vector_stores import ChromaVectorStore
db = ChromaVectorStore(collection_name="sample",embedding=embed)

2024-06-30 18:08:15,026 INFO:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [7]:
indox.connect_to_vectorstore(vectorstore_database=db)

2024-06-30 18:08:15,920 INFO:Attempting to connect to the vector store database
2024-06-30 18:08:15,921 INFO:Connection to the vector store database established successfully


In [8]:
indox.store_in_vectorstore(docs)

2024-06-30 18:08:16,497 INFO:Storing documents in the vector store
2024-06-30 18:08:22,409 INFO:Document added successfully to the vector store.
2024-06-30 18:08:22,410 INFO:Documents stored successfully


## Querying and Interpreting the Response

In this step, we query the Indox application with a specific question and use the QA model to get the response. 



In [9]:
query = "How cinderella reach her happy ending?"

In [10]:
retriever = indox.QuestionAnswer(vector_database=db,llm=google_qa,top_k=5)

In [11]:
answer = retriever.invoke(query=query)

2024-06-30 18:08:24,452 INFO:Retrieving context and scores from the vector database
2024-06-30 18:08:24,522 INFO:Generating answer without document relevancy filter
2024-06-30 18:08:24,523 INFO:Answering question: How cinderella reach her happy ending?
2024-06-30 18:08:24,523 INFO:Generating response for prompt: Given Context: ["Beautiful dresses, said one, pearls and jewels, said the second. And you, cinderella, said he, what will you have.  Father break off for me the first branch which knocks against your hat on your way home.  So he bought beautiful dresses, pearls and jewels for his two step-daughters, and on his way home, as he was riding through a green thicket, a hazel twig brushed against him and knocked off his hat.  Then he broke off the branch and took it with him.  When he reached home he gave his step-daughters the things which they had wished for, and to cinderella he gave the branch from the hazel-bush.  Cinderella thanked him, went to her mother's grave and planted the

In [12]:
answer

"Cinderella reaches her happy ending through a combination of kindness, resourcefulness, and a little bit of magic.  She uses the hazel-tree gifted by her father to communicate with a helpful bird that grants her wishes.  This bird provides her with beautiful dresses and golden slippers, allowing her to attend the king's ball and capture the prince's attention.  Despite her step-family's attempts to sabotage her, Cinderella's kindness and good nature shine through, and the prince, recognizing her true beauty, seeks her out.  The golden slipper, a symbol of her grace and charm, ultimately leads the prince to her, proving that despite adversity, true love and kindness prevail."

In [15]:
retriever.context

["Beautiful dresses, said one, pearls and jewels, said the second. And you, cinderella, said he, what will you have.  Father break off for me the first branch which knocks against your hat on your way home.  So he bought beautiful dresses, pearls and jewels for his two step-daughters, and on his way home, as he was riding through a green thicket, a hazel twig brushed against him and knocked off his hat.  Then he broke off the branch and took it with him.  When he reached home he gave his step-daughters the things which they had wished for, and to cinderella he gave the branch from the hazel-bush.  Cinderella thanked him, went to her mother's grave and planted the branch on it, and wept so much that the tears fell down on it and watered it.  And it grew and became a handsome tree. Thrice a day cinderella went and sat beneath it, and wept and prayed, and a little white bird always came on the tree, and if cinderella expressed a wish, the bird threw down to her what she had wished for. It